<h1>Data Wrangling with OpenStreetMap and SQL</h1>

<b>Name:</b> Faraz Mirza

<b>Map Area:</b> I have choosen San Francisco Bay Area as I plan to move there.

<b>Location:</b> San Francisco Bay Area, USA  

<b>OpenStreetMap URL:</b>
http://wiki.openstreetmap.org/wiki/San_Francisco_Bay_Area,_California

<b>MapZen URL:</b>
https://s3.amazonaws.com/metro-extracts.mapzen.com/san-francisco-bay_california.osm.bz2


## 1. Data Audit

### Unique Tags  

Looking at the XML file, I found that it uses different types of tags. So, I parsed the San Francisco Bay, USA dataset using ElementTree and count number of the unique tags.  
<br>
<li>'member': 222,</li>
<li>'nd': 32900</li>
<li>'node': 30456</li>
<li>'osm': 1</li>
<li>'relation': 27</li>
<li>'tag': 11542</li>
<li>'way': 3509</li>

### Patterns in the Tags

The "k" value of each tag contain different patterns. I created 3 regular expressions to check for certain patterns in the tags. I have counted each of four tag categories.  
<br>
<li>"lower" : 6873, for tags that contain only lowercase letters and are valid,</li>
<li>"lower_colon" : 4371, for otherwise valid tags with a colon in their names,</li>
<li>"two_colon" : 36, for tags that contain two colons in their names,</li>
<li>"problemchars" : 0, for tags with problematic characters, and</li>
<li>"other" : 262, for other tags that do not fall into the other four categories.</li>


## 2. Problems Encountered in the Map

### Street address inconsistencies 

The main problem we encountered in the dataset is the street name inconsistencies. Below is the old name corrected with the better name.

#### Abbreviations  

Somme Ave -> Somme Avenue  
San Felipe Rd -> San Felipe Road  
Alpine Dr -> Alpine Drive  
South St -> South Street  
Teresita Ct -> Teresita Court  

### Postal Code inconsistencies

Consistency in datasets is highly desirable, since query results will be inaccurate if the assumed types are not used with 100% consistency. In the case of Postal Codes, it was noted that the Postal Code was mostly the standard five - digit format, but in some instances, It would include the extra four-digit extension. These extensions were removed to maintain consistency by using this regex in the shape element function:  

one_hyphen = re.compile(r'^([a-z]|_|[A-Z]|[0-9])*-([a-z]|_|[A-Z]|[0-9])*$')  

#### Postal Codes  

95014-0129 -> 95014  
95023-9235 -> 95023



## 3. Data Overview and Additional Ideas

The sizes of the files used in this project are as follows:  

<li>OSM File: 3.0 GB</li>
<li>Sample File: 6.2 MB</li>
<li>Database Size: 14.5 MB</li>
<li>Nodes.csv: 2.4 MB</li>
<li>Nodes_tagss.csv: 56.6 KB</li>
<li>Ways.csv: 206.9 KB</li>
<li>Ways_tags.csv: 330.8 KB</li>
<li>Ways_nodes.csv: 763.9 KB</li>

Other statistics from the database (and the associated SQL code) now follows:

#### Number of nodes:

SELECT COUNT(*) FROM nodes;  
30456  

#### Number of ways:

SELECT COUNT(*) FROM ways;  
3509  

#### Number of unique users:

SELECT COUNT(DISTINCT(e.uid))            
FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;  
931  

#### Top 10 contributing users:  

SELECT e.user, COUNT(*) as num  
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e  
GROUP BY e.user  
ORDER BY num DESC  
LIMIT 10;  

('andygol', 3727),  
 ('nmixter', 3391),  
 ('ediyes', 2098),  
 ('Luis36995', 1607),  
 ('Eureka gold', 1565),  
 ('dannykath', 1414),  
 ('RichRico', 1303),  
 ('woodpeck_fixbot', 1185),  
 ('Rub21', 820),  
 ('mk408', 817)   
 


## Additional Ideas

### Additional Data Exploration

#### Top 10 appearing amenities

SELECT value, COUNT(*) as num  
FROM nodes_tags  
WHERE key='amenity'  
GROUP BY value  
ORDER BY num DESC  
LIMIT 10;  

('restaurant', 15),  
 ('drinking_water', 7),  
 ('fast_food', 6),  
 ('place_of_worship', 6),  
 ('cafe', 4),  
 ('post_box', 4),  
 ('atm', 3),  
 ('fire_station', 2),  
 ('fuel', 2),  
 ('school', 2)  
    
#### Most popular cuisines

SELECT nodes_tags.value, COUNT(*) as num  
FROM nodes_tags JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i ON nodes_tags.id=i.id  
WHERE nodes_tags.key='cuisine'  
GROUP BY nodes_tags.value  
ORDER BY num DESC;  

('chinese', 2),  
 ('mexican', 2),  
 ('vietnamese', 2),  
 ('brazilian', 1),  
 ('japanese', 1),  
 ('thai', 1)  

#### Biggest religion

SELECT nodes_tags.value, COUNT(*) as num  
FROM nodes_tags JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='place_of_worship') i ON nodes_tags.id=i.id  
WHERE nodes_tags.key='religion'  
GROUP BY nodes_tags.value  
ORDER BY num DESC  
LIMIT 1;  

'christian', 5


## Conclusion

After this review of the data, I can tell that the San Francisco Bay Area is incomplete. The categorization of data, auditing and cleaning has been done on basis of a sample of Original Data(3.00 GB) which might not be enough to correctly remove all kinds of problems. With a rough GPS data processor in place like pygeiocoder along with other libraries, I think it would be possible to input a great amount of cleaned data to OpenStreetMap.org. 

It interests me to notice a fair amount of GPS data makes it into OpenStreetMap.org on account of users’ efforts. There are a high number of users with more than 100 contributions. However, there's a significant number of users that have contributions lower than 10. OSM can improve these contributions if they provide a fast learning map editor with an interactive tutorial, inline help and a much more intuitive UI helping users to map hundreds of distinct features like roads, trails, buildings, parks, cafés, schools and hospitals.



### References  

[1] OpenStreetMap Sample Project Data Wrangling with SQL by Carlward  

https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md